In [ ]:
import os
import sys
from pathlib import Path

# --- Local Jupyter setup ---
# Ensure we run from the repo root and can `import src`.
repo_root = Path.cwd().resolve()
if repo_root.name == "notebooks":
    repo_root = repo_root.parent

os.chdir(repo_root)
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

print("Notebook Python:", sys.version.split()[0])
#print("Repo root:", repo_root)


Notebook Python: 3.11.14


In [3]:
# Section 2 — Load the menu index (Bootstrap)
from src.bootstrap import load_index_with_summary

index, summary = load_index_with_summary("data/dataset.json")
summary

{'total_items': 46,
 'total_categories': 7,
 'total_discounts': 9,
 'has_channel_pricing': False,
 'notes': ['No channel-specific price overrides detected in dataset']}

## Trace inspection

#### LLM router - smaller model

In [4]:
from src.chat import answer

questions = [
    "What is the price of a small NUTTY BOWL?",
    "Which discounts include coupons?",
    "What items trigger BOGO Any Smoothie discount?",
]

for q in questions:
    print("Q:", q)
    print("A:", answer(q, index, debug=True))
    print("-" * 80)


Q: What is the price of a small NUTTY BOWL?


[trace] router.result {"router": "llm", "model": "gpt-4o-mini", "reason": null, "error_type": null, "intent": "get_price", "entities": {"item": "NUTTY BOWL", "portion": "small", "category": null, "discount": null, "channel": null}, "raw_llm_output_preview": "{\n  \"intent\": \"get_price\",\n  \"item\": \"NUTTY BOWL\",\n  \"portion\": \"small\",\n  \"category\": null,\n  \"discount\": null,\n  \"channel\": null\n}"}
[trace] router.postprocess {"coupon_override_applied": false, "discount_sanitized": false, "intent_before": "get_price", "intent_after": "get_price", "discount_before": null, "discount_after": null}
[trace] resolver.item {"query": "NUTTY BOWL", "normalized_query": "nutty bowl", "ok": true, "reason": "exact", "resolved_id": 74164, "resolved_display": "NUTTY BOWL", "candidates": []}
[trace] tool.result {"tool": "get_item_price", "ok": true, "error_code": null, "candidate_count": 0, "meta_keys": ["portion_normalized", "resolved_item_id", "resolved_item_name"]}


A: $11.99 — NUTTY BOWL (Small)
--------------------------------------------------------------------------------
Q: Which discounts include coupons?


[trace] router.result {"router": "llm", "model": "gpt-4o-mini", "reason": null, "error_type": null, "intent": "discount_details", "entities": {"item": null, "portion": null, "category": null, "discount": "coupons", "channel": null}, "raw_llm_output_preview": "{\n  \"intent\": \"discount_details\",\n  \"item\": null,\n  \"portion\": null,\n  \"category\": null,\n  \"discount\": \"coupons\",\n  \"channel\": null\n}"}
[trace] router.postprocess {"coupon_override_applied": true, "discount_sanitized": true, "intent_before": "discount_details", "intent_after": "discount_details", "discount_before": "coupons", "discount_after": null}


A: Discounts with coupons (3): $3.00 Off LG Bowl, Military/Police Discount, Open Discount
--------------------------------------------------------------------------------
Q: What items trigger BOGO Any Smoothie discount?
A: BOGO Any Smoothie triggers: ACAI ELIXIR, DRAGON SMOOTHIE, GO GREEN, KID SUNSATION, MATCHA MADNESS, TEMPTATION, TROPICAL PARADISE, WHEY GREEN
--------------------------------------------------------------------------------


[trace] router.result {"router": "llm", "model": "gpt-4o-mini", "reason": null, "error_type": null, "intent": "discount_triggers", "entities": {"item": "Smoothie", "portion": null, "category": null, "discount": "BOGO", "channel": null}, "raw_llm_output_preview": "{\n  \"intent\": \"discount_triggers\",\n  \"item\": \"Smoothie\",\n  \"portion\": null,\n  \"category\": null,\n  \"discount\": \"BOGO\",\n  \"channel\": null\n}"}
[trace] router.postprocess {"coupon_override_applied": false, "discount_sanitized": true, "intent_before": "discount_triggers", "intent_after": "discount_triggers", "discount_before": "BOGO", "discount_after": "bogo any smoothie"}
[trace] resolver.discount {"query": "bogo any smoothie", "normalized_query": "bogo any smoothie", "ok": true, "reason": "exact", "resolved_id": 79376, "resolved_display": "BOGO Any Smoothie", "candidates": []}
[trace] tool.result {"tool": "discount_triggers", "ok": true, "error_code": null, "candidate_count": 0, "meta_keys": ["item_group_

### LLM router - stronger model

In [5]:
os.environ["OPENAI_MODEL"] = "gpt-5.1"

from src.chat import answer

questions = [
    "What is the price of a small NUTTY BOWL?",
    "Which discounts include coupons?",
    "What items trigger BOGO Any Smoothie discount?",
]

for q in questions:
    print("Q:", q)
    print("A:", answer(q, index, debug=True))
    print("-" * 80)


Q: What is the price of a small NUTTY BOWL?


[trace] router.result {"router": "llm", "model": "gpt-5.1", "reason": null, "error_type": null, "intent": "get_price", "entities": {"item": "NUTTY BOWL", "portion": "small", "category": null, "discount": null, "channel": null}, "raw_llm_output_preview": "{\n  \"intent\": \"get_price\",\n  \"item\": \"NUTTY BOWL\",\n  \"portion\": \"small\",\n  \"category\": null,\n  \"discount\": null,\n  \"channel\": null\n}"}
[trace] router.postprocess {"coupon_override_applied": false, "discount_sanitized": false, "intent_before": "get_price", "intent_after": "get_price", "discount_before": null, "discount_after": null}
[trace] resolver.item {"query": "NUTTY BOWL", "normalized_query": "nutty bowl", "ok": true, "reason": "exact", "resolved_id": 74164, "resolved_display": "NUTTY BOWL", "candidates": []}
[trace] tool.result {"tool": "get_item_price", "ok": true, "error_code": null, "candidate_count": 0, "meta_keys": ["portion_normalized", "resolved_item_id", "resolved_item_name"]}


A: $11.99 — NUTTY BOWL (Small)
--------------------------------------------------------------------------------
Q: Which discounts include coupons?


[trace] router.result {"router": "llm", "model": "gpt-5.1", "reason": null, "error_type": null, "intent": "discount_details", "entities": {"item": null, "portion": null, "category": null, "discount": "coupons", "channel": null}, "raw_llm_output_preview": "{\n  \"intent\": \"discount_details\",\n  \"item\": null,\n  \"portion\": null,\n  \"category\": null,\n  \"discount\": \"coupons\",\n  \"channel\": null\n}"}
[trace] router.postprocess {"coupon_override_applied": true, "discount_sanitized": true, "intent_before": "discount_details", "intent_after": "discount_details", "discount_before": "coupons", "discount_after": null}


A: Discounts with coupons (3): $3.00 Off LG Bowl, Military/Police Discount, Open Discount
--------------------------------------------------------------------------------
Q: What items trigger BOGO Any Smoothie discount?
A: BOGO Any Smoothie triggers: ACAI ELIXIR, DRAGON SMOOTHIE, GO GREEN, KID SUNSATION, MATCHA MADNESS, TEMPTATION, TROPICAL PARADISE, WHEY GREEN
--------------------------------------------------------------------------------


[trace] router.result {"router": "llm", "model": "gpt-5.1", "reason": null, "error_type": null, "intent": "discount_triggers", "entities": {"item": null, "portion": null, "category": "smoothie", "discount": "BOGO Any Smoothie", "channel": null}, "raw_llm_output_preview": "{\n  \"intent\": \"discount_triggers\",\n  \"item\": null,\n  \"portion\": null,\n  \"category\": \"smoothie\",\n  \"discount\": \"BOGO Any Smoothie\",\n  \"channel\": null\n}"}
[trace] router.postprocess {"coupon_override_applied": false, "discount_sanitized": false, "intent_before": "discount_triggers", "intent_after": "discount_triggers", "discount_before": "BOGO Any Smoothie", "discount_after": "BOGO Any Smoothie"}
[trace] resolver.discount {"query": "BOGO Any Smoothie", "normalized_query": "bogo any smoothie", "ok": true, "reason": "exact", "resolved_id": 79376, "resolved_display": "BOGO Any Smoothie", "candidates": []}
[trace] tool.result {"tool": "discount_triggers", "ok": true, "error_code": null, "candidate_c

### Fallback router - rules-based

In [6]:
_ = os.environ.pop("OPENAI_API_KEY", None)

In [7]:
for q in questions:
    print("Q:", q)
    print("A:", answer(q, index, debug=True))
    print("-" * 80)

Q: What is the price of a small NUTTY BOWL?
A: $11.99 — NUTTY BOWL (Small)
--------------------------------------------------------------------------------
Q: Which discounts include coupons?
A: Discounts with coupons (3): $3.00 Off LG Bowl, Military/Police Discount, Open Discount
--------------------------------------------------------------------------------
Q: What items trigger BOGO Any Smoothie discount?
A: BOGO Any Smoothie triggers: ACAI ELIXIR, DRAGON SMOOTHIE, GO GREEN, KID SUNSATION, MATCHA MADNESS, TEMPTATION, TROPICAL PARADISE, WHEY GREEN
--------------------------------------------------------------------------------


[trace] router.result {"router": "fallback", "model": null, "reason": "missing_api_key", "error_type": null, "intent": "get_price", "entities": {"item": "nutty bowl", "portion": "small", "category": null, "discount": null, "channel": null}, "raw_llm_output_preview": null}
[trace] router.postprocess {"coupon_override_applied": false, "discount_sanitized": false, "intent_before": "get_price", "intent_after": "get_price", "discount_before": null, "discount_after": null}
[trace] resolver.item {"query": "nutty bowl", "normalized_query": "nutty bowl", "ok": true, "reason": "exact", "resolved_id": 74164, "resolved_display": "NUTTY BOWL", "candidates": []}
[trace] tool.result {"tool": "get_item_price", "ok": true, "error_code": null, "candidate_count": 0, "meta_keys": ["portion_normalized", "resolved_item_id", "resolved_item_name"]}
[trace] router.result {"router": "fallback", "model": null, "reason": "missing_api_key", "error_type": null, "intent": "discount_details", "entities": {"item": nul